# golub dataset

In [ ]:
# install.packages("binom") # Installed from R studio as needs compilation
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("cancerclass")

In [34]:
library("ISLR") 
library("evaluomeR")
library("dplyr")

options(scipen=10)

In [ ]:
# https://rdrr.io/bioc/cancerclass/man/GOLUB.html

In [35]:
nci60 = as.data.frame(NCI60$data)

In [36]:
head(nci60)
print(paste0("Rows: ", nrow(nci60)))
print(paste0("Columns: ", ncol(nci60)))

,1,2,3,4,5,6,7,8,9,10,...,6821,6822,6823,6824,6825,6826,6827,6828,6829,6830
V1,0.300000,1.180000,0.550000,1.140000,-0.265000,-7.000000e-02,0.350000,-0.315000,-0.45000000,-0.65498050,...,-0.9900195,0.000000,0.030000,-0.175000,0.6299805,-0.030000,0.0000000,0.280000,-0.340000,-1.930000
V2,0.679961,1.289961,0.169961,0.379961,0.464961,5.799610e-01,0.699961,0.724961,-0.04003899,-0.28501950,...,-0.2700585,-0.300039,-0.250039,-0.535039,0.1099415,-0.860039,-1.2500490,-0.770039,-0.390039,-2.000039
V3,0.940000,-0.040000,-0.170000,-0.040000,-0.605000,0.000000e+00,0.090000,0.645000,0.43000000,0.47501950,...,0.3199805,0.120000,-0.740000,-0.595000,-0.2700195,-0.150000,0.0000000,-0.120000,-0.410000,0.000000
V4,0.280000,-0.310000,0.680000,-0.810000,0.625000,-1.387779e-17,0.170000,0.245000,0.02000000,0.09501949,...,-1.2400200,-0.110000,-0.160000,0.095000,-0.3500195,-0.300000,-1.1500100,1.090000,-0.260000,-1.100000
V5,0.485000,-0.465000,0.395000,0.905000,0.200000,-5.000000e-03,0.085000,0.110000,0.23500000,1.49001949,...,0.5549805,-0.775000,-0.515000,-0.320000,0.6349805,0.605000,0.0000000,0.745000,0.425000,0.145000
V6,0.310000,-0.030000,-0.100000,-0.460000,-0.205000,-5.400000e-01,-0.640000,-0.585000,-0.77000000,-0.24498050,...,-0.5900195,0.000000,-0.740000,-2.095000,-1.3900200,-0.400000,0.6999902,0.290000,0.550000,0.000000


[1] "Rows: 64"
[1] "Columns: 6830"


In [37]:
as.data.frame(unique(NCI60$labs))

unique(NCI60$labs)
CNS
RENAL
BREAST
NSCLC
UNKNOWN
OVARIAN
MELANOMA
PROSTATE
LEUKEMIA
K562B-repro


In [38]:
nci60["labels"] = rownames(nci60)
nci60 = nci60[ , c("labels", names(nci60)[names(nci60) != "labels"])]
nci60["labels"] = NCI60$labs
colnames(nci60)[colnames(nci60) == 'labels'] <- 'Description'
nci60 = nci60[!grepl("UNKNOWN", nci60$Description),] # Remove UNKNOWN
#nci60 = nci60[!grepl("PROSTATE", nci60$Description),] # Remove PROSTATE

In [39]:
unique(nci60$Description)
head(nci60)

[1] "CNS"         "RENAL"       "BREAST"      "NSCLC"       "OVARIAN"    
 [6] "MELANOMA"    "PROSTATE"    "LEUKEMIA"    "K562B-repro" "K562A-repro"
[11] "COLON"       "MCF7A-repro" "MCF7D-repro"

,Description,1,2,3,4,5,6,7,8,9,...,6821,6822,6823,6824,6825,6826,6827,6828,6829,6830
V1,CNS,0.300000,1.180000,0.550000,1.140000,-0.265000,-7.000000e-02,0.350000,-0.315000,-0.45000000,...,-0.9900195,0.000000,0.030000,-0.175000,0.6299805,-0.030000,0.0000000,0.280000,-0.340000,-1.930000
V2,CNS,0.679961,1.289961,0.169961,0.379961,0.464961,5.799610e-01,0.699961,0.724961,-0.04003899,...,-0.2700585,-0.300039,-0.250039,-0.535039,0.1099415,-0.860039,-1.2500490,-0.770039,-0.390039,-2.000039
V3,CNS,0.940000,-0.040000,-0.170000,-0.040000,-0.605000,0.000000e+00,0.090000,0.645000,0.43000000,...,0.3199805,0.120000,-0.740000,-0.595000,-0.2700195,-0.150000,0.0000000,-0.120000,-0.410000,0.000000
V4,RENAL,0.280000,-0.310000,0.680000,-0.810000,0.625000,-1.387779e-17,0.170000,0.245000,0.02000000,...,-1.2400200,-0.110000,-0.160000,0.095000,-0.3500195,-0.300000,-1.1500100,1.090000,-0.260000,-1.100000
V5,BREAST,0.485000,-0.465000,0.395000,0.905000,0.200000,-5.000000e-03,0.085000,0.110000,0.23500000,...,0.5549805,-0.775000,-0.515000,-0.320000,0.6349805,0.605000,0.0000000,0.745000,0.425000,0.145000
V6,CNS,0.310000,-0.030000,-0.100000,-0.460000,-0.205000,-5.400000e-01,-0.640000,-0.585000,-0.77000000,...,-0.5900195,0.000000,-0.740000,-2.095000,-1.3900200,-0.400000,0.6999902,0.290000,0.550000,0.000000


In [ ]:
#nrow(nci60[nci60$Description == "PROSTATE", ])

In [40]:
variance = sort(sapply(nci60[-1], var), decreasing = TRUE)  # Sorted gene variance
nci60_var = as.data.frame(variance)
nci60_var["Description"] = rownames(nci60_var)

In [41]:
top_number = 200
top_rows = nci60_var[c(1:top_number), ]
head(top_rows)

,variance,Description
4701,11.713375,4701
4700,11.301019,4700
6393,7.916659,6393
256,7.890767,256
4699,7.602618,4699
16,7.523190,16


In [42]:
row_list = as.list(top_rows["Description"])

In [43]:
top_row_list = unlist(setdiff(row_list, names(nci60)))
top_row_list

[1] "4701" "4700" "6393" "256"  "4699" "16"   "286"  "6415" "6149" "4354"
 [11] "113"  "6391" "257"  "252"  "6150" "5942" "6151" "5557" "196"  "4353"
 [21] "243"  "6392" "6152" "5805" "5980" "5556" "5948" "5732" "5586" "3957"
 [31] "5937" "4344" "2068" "5869" "6416" "4094" "755"  "5868" "134"  "6279"
 [41] "5760" "4154" "112"  "4288" "6278" "5943" "5804" "6156" "5692" "6087"
 [51] "2082" "224"  "6688" "4308" "4716" "6635" "3518" "2074" "2080" "4244"
 [61] "6277" "5705" "1390" "5979" "248"  "2081" "5555" "3525" "6412" "3936"
 [71] "4093" "5838" "754"  "975"  "1391" "4425" "4060" "133"  "5803" "5587"
 [81] "281"  "6689" "4280" "301"  "4426" "6084" "5910" "6430" "2083" "4085"
 [91] "5275" "5946" "3248" "6429" "4057" "1389" "5917" "4289" "6710" "5796"
[101] "287"  "4388" "6274" "5845" "5940" "5828" "5142" "6612" "4306" "6717"
[111] "3543" "5706" "4304" "5472" "188"  "267"  "5870" "3438" "6646" "5884"
[121] "6128" "5707" "5988" "5984" "4327" "6153" "6157" "5972" "6148" "5353"
[131] "5867" "1393" "5276" "251"  "5496" "6413" "5758" "5829" "4383" "5477"
[141] "6264" "6356" "5510" "5481" "5729" "5588" "5981" "242"  "1388" "6419"
[151] "6644" "5417" "1120" "5712" "5691" "5830" "273"  "266"  "3894" "5836"
[161] "5646" "3706" "3956" "4010" "5336" "5730" "6154" "5973" "111"  "6243"
[171] "6404" "247"  "5843" "5913" "5561" "756"  "6086" "78"   "3490" "5916"
[181] "4387" "6288" "229"  "5661" "5721" "6414" "5489" "245"  "6272" "5631"
[191] "5872" "6622" "6564" "1012" "4845" "6294" "1067" "5899" "124"  "3383"

In [44]:
top_nci60 = nci60[, top_row_list]
#top_nci60["Description"] = rownames(top_nci60)
top_nci60["Description"] = nci60[rownames(top_nci60) %in% rownames(nci60), "Description"]
top_nci60 = top_nci60[ , c("Description", names(top_nci60)[names(top_nci60) != "Description"])] 

In [45]:
head(top_nci60)

,Description,4701,4700,6393,256,4699,16,286,6415,6149,...,5872,6622,6564,1012,4845,6294,1067,5899,124,3383
V1,CNS,5.770000,5.820000,5.4799900,-1.910000,4.760000,-2.37,0.610000,5.230000,0.000000,...,0.045000,1.210000,-1.3049900,0.000000,0.00000000,1.250000,-1.055010,1.255000,0.000000,-0.5400195
V2,CNS,5.039961,5.069961,5.7299510,-1.980039,4.779961,0.00,-3.980039,5.329961,-0.480039,...,0.874961,1.599961,-0.3750292,-1.870039,0.23498050,0.959961,-1.055049,1.604961,-1.015039,0.4799415
V3,CNS,-1.440000,-1.030000,4.6299900,-0.580000,0.000000,0.00,-1.580000,0.000000,2.060000,...,1.445000,-0.260000,1.0250100,0.000000,-0.06498051,-0.820000,1.294990,3.365000,0.000000,0.3199805
V4,RENAL,-2.420000,-2.420000,1.2999900,-1.880000,-2.590000,-1.02,-4.880000,7.020000,5.190000,...,2.365000,1.110000,1.3150100,2.430000,0.39501950,-0.670000,1.414990,3.725000,0.000000,0.1399805
V5,BREAST,-2.955000,-2.955000,-1.0550098,-1.235000,-2.535000,0.00,-3.235000,4.395000,7.425000,...,3.320000,1.245000,-0.6299902,0.925000,-0.64998051,-0.795000,-1.480010,2.530000,0.000000,0.4449805
V6,CNS,0.000000,-3.180000,0.5899902,-3.050000,-1.170000,-0.05,-5.050000,-0.220000,4.230000,...,1.615000,1.360000,-0.8549902,0.700000,1.44501900,-0.640000,-1.425010,3.365000,0.725000,-1.5400200


# evaluomeR K analysis

In [58]:
seed = 13606
k.range=c(3,10)
cbi = "kmeans"

# Stab
stab_range = stabilityRange(data=top_nci60, k.range=k.range, 
                            bs=100, seed=seed,
                            all_metrics=TRUE,
                            cbi=cbi)
stab = standardizeStabilityData(stab_range)

# Qual
qual_range = qualityRange(data=top_nci60, k.range=k.range, 
                            seed=seed,
                            all_metrics=TRUE,
                            cbi=cbi)
qual = standardizeQualityData(qual_range)

# K opt
k_opt = getOptimalKValue(stab_range, qual_range, k.range= k.range)


Data loaded.
Number of rows: 63
Number of columns: 201


Processing all metrics, 'merge', in dataframe (200)
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6
	Calculation of k = 7
	Calculation of k = 8
	Calculation of k = 9
	Calculation of k = 10

Data loaded.
Number of rows: 63
Number of columns: 201


Processing all metrics, 'merge', in dataframe (200)
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6
	Calculation of k = 7
	Calculation of k = 8
	Calculation of k = 9
	Calculation of k = 10
Processing metric: all_metrics

	Both Ks do not have a stable classification: '4', '9'

	Using '9' since it provides higher silhouette width



In [59]:
print(paste0("W/ ", cbi))
stab
qual

[1] "W/ kmeans"


,k_3,k_4,k_5,k_6,k_7,k_8,k_9,k_10
all_metrics,0.5965715,0.7003381,0.5702042,0.6104274,0.6044109,0.5989443,0.6386568,0.6071409


,k_3,k_4,k_5,k_6,k_7,k_8,k_9,k_10
all_metrics,0.1302571,0.1607795,0.1651107,0.1558715,0.1566324,0.1546223,0.1672433,0.1664928


# Clusters

In [15]:
individuals_per_cluster = function(qualityResult) {
  qual_df = as.data.frame(assay(qualityResult))


  cluster_pos_str = as.character(unlist(qual_df["Cluster_position"]))
  cluster_labels_str = as.character(unlist(qual_df["Cluster_labels"]))

  cluster_pos = as.list(strsplit(cluster_pos_str, ",")[[1]])
  cluster_labels = as.list(strsplit(cluster_labels_str, ",")[[1]])

  individuals_in_cluster = as.data.frame(cbind(cluster_labels, cluster_pos))
  colnames(individuals_in_cluster) = c("Individual", "InCluster")

  return(individuals_in_cluster)
}

In [57]:
qual_k10_kmeans = qualityRange(data=top_nci60, k.range=k.range, 
                            seed=seed,
                            all_metrics=TRUE,
                            cbi="kmeans")
nci60_k = individuals_per_cluster(assay(qual_k10_kmeans$k_8))
print("Kmeans k10")
standardizeQualityData(qual_k10_kmeans)
for (cluster_i in 1:10) {
    ind_in_cluster = paste(unlist(nci60_k[nci60_k$InCluster == cluster_i, ]["Individual"]), collapse = ",")
    print(paste("Cluster", cluster_i, ":", ind_in_cluster))
    print("---")
}


Data loaded.
Number of rows: 63
Number of columns: 201


Processing all metrics, 'merge', in dataframe (200)
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6
	Calculation of k = 7
	Calculation of k = 8
	Calculation of k = 9
	Calculation of k = 10


[1] "Kmeans k10"


,k_3,k_4,k_5,k_6,k_7,k_8,k_9,k_10
all_metrics,0.1302571,0.1607795,0.1651107,0.1558715,0.1566324,0.1546223,0.1672433,0.1664928


[1] "Cluster 1 : MCF7A-repro,BREAST,MCF7D-repro,BREAST"
[1] "---"
[1] "Cluster 2 : CNS,CNS,CNS,RENAL,BREAST,CNS,CNS,BREAST,NSCLC,NSCLC,BREAST,NSCLC,RENAL,OVARIAN,MELANOMA"
[1] "---"
[1] "Cluster 3 : K562B-repro,K562A-repro,LEUKEMIA"
[1] "---"
[1] "Cluster 4 : RENAL,RENAL,RENAL,RENAL,RENAL,RENAL,RENAL"
[1] "---"
[1] "Cluster 5 : LEUKEMIA,LEUKEMIA,LEUKEMIA,LEUKEMIA"
[1] "---"
[1] "Cluster 6 : MELANOMA,BREAST,BREAST,MELANOMA,MELANOMA,MELANOMA,MELANOMA,MELANOMA,MELANOMA"
[1] "---"
[1] "Cluster 7 : PROSTATE,OVARIAN,PROSTATE,COLON,COLON,COLON,COLON,COLON,COLON,COLON,NSCLC,NSCLC"
[1] "---"
[1] "Cluster 8 : OVARIAN,OVARIAN,OVARIAN,OVARIAN,NSCLC,NSCLC,NSCLC,LEUKEMIA,NSCLC"
[1] "---"
[1] "Cluster 9 : "
[1] "---"
[1] "Cluster 10 : "
[1] "---"


In [53]:
qual_k8_clara = qualityRange(data=top_nci60, k.range=k.range, 
                            seed=seed,
                            all_metrics=TRUE,
                            cbi="clara")
nci60_k = individuals_per_cluster(assay(qual_k8_clara$k_6))
print("Clara k3")
standardizeQualityData(qual_k8_clara)
for (cluster_i in 1:max(unlist(unique(nci60_k["InCluster"])))) {
    ind_in_cluster = paste(unlist(nci60_k[nci60_k$InCluster == cluster_i, ]["Individual"]), collapse = ",")
    print(paste("Cluster", cluster_i, ":", ind_in_cluster))
    print("---")
}


Data loaded.
Number of rows: 63
Number of columns: 201


Processing all metrics, 'merge', in dataframe (200)
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6
	Calculation of k = 7
	Calculation of k = 8
	Calculation of k = 9
	Calculation of k = 10


[1] "Clara k3"


,k_3,k_4,k_5,k_6,k_7,k_8,k_9,k_10
all_metrics,0.1307175,0.1472834,0.1450821,0.143228,0.1526406,0.1316738,0.1372842,0.1350317


[1] "Cluster 1 : CNS,CNS,CNS,RENAL,BREAST,CNS,CNS,BREAST,NSCLC,NSCLC"
[1] "---"
[1] "Cluster 2 : RENAL,RENAL,RENAL,RENAL,RENAL,RENAL,RENAL,NSCLC,OVARIAN,NSCLC,NSCLC,NSCLC"
[1] "---"
[1] "Cluster 3 : BREAST,RENAL,MELANOMA,PROSTATE,OVARIAN,OVARIAN,OVARIAN,OVARIAN,OVARIAN,PROSTATE,COLON,COLON,COLON,COLON,COLON,COLON,COLON,NSCLC,NSCLC,NSCLC"
[1] "---"
[1] "Cluster 4 : LEUKEMIA,MELANOMA,BREAST,BREAST,MELANOMA,MELANOMA,MELANOMA,MELANOMA,MELANOMA,MELANOMA"
[1] "---"
[1] "Cluster 5 : K562B-repro,K562A-repro,LEUKEMIA,LEUKEMIA,LEUKEMIA,LEUKEMIA,LEUKEMIA"
[1] "---"
[1] "Cluster 6 : MCF7A-repro,BREAST,MCF7D-repro,BREAST"
[1] "---"
